[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_diffusers_to_sd.ipynb)

In [ ]:
!pip -qq install huggingface-hub
!wget -q https://gist.github.com/jachiam/8a5c0b607e38fcc585168b90c686eb05/raw/2af0a9c0237ed98b863a75e1db21d7ed5541094f/convert_diffusers_to_sd.py
!git lfs install
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

In [ ]:
model_name = "modelname" #@param {type:"string"}
!git clone https://huggingface.co/camenduru/$model_name

model_name_ckpt = f"/content/{model_name}_ckpt"
!mkdir $model_name_ckpt

ckpt_path = f"/content/{model_name}_ckpt/{model_name}.ckpt"
!python convert_diffusers_to_sd.py --model_path /content/$model_name --checkpoint_path $ckpt_path 

ckpt_path = f"/content/{model_name}_ckpt/{model_name}_fp16.ckpt"
!python convert_diffusers_to_sd.py --model_path /content/$model_name --checkpoint_path $ckpt_path --half

from slugify import slugify
from huggingface_hub import HfApi, HfFolder, CommitOperationAdd
from huggingface_hub import create_repo
from IPython.display import display_markdown
api = HfApi()
your_username = api.whoami()["name"]
repo_id = f"{your_username}/{slugify(model_name)}_ckpt"
with open(HfFolder.path_token, 'r') as fin: hf_token = fin.read();

operations = []
create_repo(repo_id,private=True, token=hf_token)
api.create_commit(repo_id=repo_id,operations=operations,commit_message=f"{model_name}",token=hf_token)
api.upload_folder(folder_path=f"/content/{model_name}_ckpt",path_in_repo="",repo_id=repo_id,token=hf_token)

display_markdown(f'https://huggingface.co/{repo_id}', raw=True)